In [ ]:
from msibi import MSIBI, State, Pair, Bond, Angle
from cmeutils.structure import angle_distribution, bond_distribution
import gsd
import gsd.hoomd
import time
import matplotlib.pyplot as plt
import math
import numpy as np

In [ ]:
%%bash

for dir in "states" "rdfs" "potentials"
do
    if [ -d $dir ]
    then
        rm -r $dir
    fi
done

In [ ]:
cg_traj_files = [
    "4.1kT.gsd",
    "5.5kT.gsd",
    "6.5kT.gsd"
]

cg_traj_files2 = [
    "4.1kT-1.27den.gsd",
]

In [ ]:
opt = MSIBI(
    integrator="hoomd.md.integrate.nvt",
    integrator_kwargs={"tau": 0.1},
    dt=0.001,
    gsd_period=10000,
    n_steps=4e6,
    max_frames=300
)

## Add States

In [ ]:
opt.add_state(
    State(name="A", kT=4.1, traj_file=cg_traj_files[0], alpha=1.0)
)
#opt.add_state(
#    State(name="B", kT=5.5, traj_file=cg_traj_files[1], alpha=0.40)
#)
#opt.add_state(
#    State(name="C", kT=6.2, traj_file=cg_traj_files[2], alpha=0.30)
#)

## Add Pairs

In [ ]:
pair0 = Pair(type1="E", type2="E")
pair1 = Pair(type1="K", type2="K")
pair2 = Pair(type1="E", type2="K")
for pair in [pair0, pair1, pair2]:
    pair.set_table_potential(epsilon=1, sigma=1, r_min=1e-4, r_max=6.0, n_points=101)
#    pair.set_gauss(epsilon=1, sigma=1.0, r_cut=3.5)
#    pair.set_lj(epsilon=1, sigma=1, r_cut=2.5)
#    pair.set_from_file(
#        file_path=f"/home/chris/cme/forks/msibi/msibi/tutorials/peek/previous_runs/pair_pot.{pair.name}.txt"
#)
    opt.add_pair(pair)

## Add Bonds

In [ ]:
bond0 = Bond(type1="E", type2="K", head_correction_form="exponential")
bond1 = Bond(type1="K", type2="K", head_correction_form="exponential")

bond0.set_quadratic(k2=50, l0=1.51, k3=0, k4=0, l_min=0.0, l_max=4.0, n_points=100)
bond1.set_quadratic(k2=50, l0=1.55, k3=0, k4=0, l_min=0.0, l_max=4.0, n_points=100)
#bond0.set_harmonic(k=50, l0=1.5)
#bond1.set_harmonic(k=50, l0=1.5)
#bond0.set_from_file(file_path="/home/chris/cme/forks/msibi/msibi/tutorials/peek/previous_runs/bond_pot.E-K.txt")
#bond1.set_from_file(file_path="/home/chris/cme/forks/msibi/msibi/tutorials/peek/previous_runs/bond_pot.K-K.txt")

opt.add_bond(bond0)
opt.add_bond(bond1)

## Add Angles

In [ ]:
angle0 = Angle(type1="E", type2="K", type3="K")
angle1 = Angle(type1="K", type2="E", type3="K")

angle0.set_harmonic(k=50, theta0=1.2)
angle1.set_harmonic(k=50, theta0=1.2)
#angle0.set_from_file(file_path="/home/chris/cme/forks/msibi/msibi/tutorials/peek/previous_runs/angle_pot.E-K-K.txt")
#angle1.set_from_file(file_path="/home/chris/cme/forks/msibi/msibi/tutorials/peek/previous_runs/angle_pot.K-E-K.txt")

#angle0.set_quadratic(k2=100, theta0=2.5, k3=0, k4=0, n_points=100)
#angle1.set_quadratic(k2=100, theta0=2.5, k3=0, k4=0, n_points=100)

opt.add_angle(angle0)
opt.add_angle(angle1)

## Run Pair Optimization Iterations

In [ ]:
opt.optimize_bonds(n_iterations=2)

In [ ]:
bond = opt.bonds[0]
bond2 = opt.bonds[1]

#pair = opt.pairs[0]

#angle = opt.angles[0]
#angle2 = opt.angles[1]

stateA = opt.states[0]

In [ ]:
thing = bond
plt.plot(
    thing._states[stateA]["target_distribution"][:,0],
    thing._states[stateA]["target_distribution"][:,1],
    label="target"
)

plt.plot(
    thing._states[stateA]["current_distribution"][:,0],
    thing._states[stateA]["current_distribution"][:,1],
    label="ibi"
)

plt.legend()
plt.title(thing.name)

In [ ]:
plt.plot(bond2.l_range, bond2.potential)
#plt.xlim(1.5, 1.8)

In [ ]:
thing = pair
plt.plot(
    thing._states[stateA]["target_rdf"][:,0],
    thing._states[stateA]["target_rdf"][:,1],
    label="target"
)

plt.plot(
    thing._states[stateA]["current_rdf"][:,0],
    thing._states[stateA]["current_rdf"][:,1],
    label="ibi"
)

plt.legend()
plt.title(thing.name)

In [ ]:
plt.plot(pair0.r_range, pair0.potential, "-", label=pair0.name)
plt.plot(pair1.r_range, pair1.potential, "-", label=pair1.name)
plt.plot(pair2.r_range, pair2.potential, "-", label=pair2.name)

plt.ylim(-2, 4)
plt.xlim(0, 4.0)
plt.legend()

In [ ]:
%matplotlib inline
for step in range(0, opt.n_iterations):
    plt.ylim([-1.5,3])
    plt.xlim([0.5,5])
    plt.title("MSIBI potentials by iteration")

    step_pot = np.loadtxt(f'potentials/step{step}.pot.P-M.txt')
    plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')
    
plt.title("PM Pair")
#plt.legend()
plt.show()

for step in range(0, opt.n_iterations):
    plt.ylim([-1.5,3])
    plt.xlim([0.5,5])
    plt.title("MSIBI potentials by iteration")

    step_pot = np.loadtxt(f'potentials/step{step}.pot.P-P.txt')
    plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')

plt.title("PP Pair")
#plt.legend()
plt.show()

for step in range(0, opt.n_iterations):
    plt.ylim([-1.5,3])
    plt.xlim([0.5,5])
    plt.title("MSIBI potentials by iteration")

    step_pot = np.loadtxt(f'potentials/step{step}.pot.M-M.txt')
    plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')

plt.title("MM Pair")
#plt.legend()
plt.show()

In [ ]:
%matplotlib inline
for step in range(opt.n_iterations - 1, opt.n_iterations):
    plt.ylim([-1.5,4])
    plt.xlim([0,5])
    plt.title("MSIBI potentials by iteration")

    step_pot = np.loadtxt(f'potentials/step{step}.pot.P-M.txt')
    plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')
    
plt.title("PM Pair")
#plt.legend()
plt.show()

for step in range(opt.n_iterations - 1, opt.n_iterations):
    plt.ylim([-1.5,4])
    plt.xlim([0,5])
    plt.title("MSIBI potentials by iteration")

    step_pot = np.loadtxt(f'potentials/step{step}.pot.P-P.txt')
    plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')

plt.title("PP Pair")
#plt.legend()
plt.show()

for step in range(opt.n_iterations - 1, opt.n_iterations):
    plt.ylim([-1.5,4])
    plt.xlim([0,5])
    plt.title("MSIBI potentials by iteration")

    step_pot = np.loadtxt(f'potentials/step{step}.pot.M-M.txt')
    plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')

plt.title("MM Pair")
#plt.legend()
plt.show()